# Lab 2: Attitude Estimation using IMU
- **Assigned:** Wednesday, April 12
- **Due:** Friday, April 21 @ 5pm

The purpose of this lab is:
1. To learn about accelerometers and gyroscopes, their measurement models and strengths and weaknesses
2. To implement a linear Kalman filter and analyze its properties
3. To estimate the attitude of the esp32imu cube

In [ ]:
%matplotlib inline
import matplotlib, copy
import matplotlib.pyplot as plt
from matplotlib import cm
#from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import gridspec
from matplotlib.cm import get_cmap

import numpy as np
from numpy.random import randn
from numpy import eye, array, asarray, exp

from math import sqrt
from scipy.linalg import expm
from scipy.linalg import solve_discrete_lyapunov
from scipy.linalg import sqrtm
from scipy import linalg as la
from scipy.integrate import odeint

import sympy as sym

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets 
from IPython.display import display

float_formatter = "{:.4f}".format
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})
np.set_printoptions(precision=3)
plt.rcParams["font.serif"] = "cmr12"
plt.rcParams['savefig.dpi'] = 300
plt.rcParams["figure.dpi"] = 100

## Introduction: Kalman Filtering for Attitude Estimation

An IMU has x-y-z accelerometers (measuring linear acceleration) and x-y-z gyroscopes (measuring angular velocity). Using these measurements, we would like to measure the tilt angle of the teensyimu board. Here, tilt refers to the roll (rotation about the x axis) and pitch (rotation about the y axis). Yaw (rotation about the z axis) is unobservable when only given a gyro and an accelerometer, as we will discuss below.

A naive solution for recovering tilt angle is to integrate the gyroscope measurements over time. However, noise and biases in the gyro measuring process would quickly cause this integration to diverge. Alternatively, accelerometer measurements can provide a measure of the angles, but these measurements are noisy and become distorted for  an accelerated body. By *fusing* these measurement sources together in a Kalman filter, we can leverage the best of both measurement sources.

Note that for simplicity, our solution will only be valid for small tilt angles, less than $\pm90$ deg in each axis.

<br />
<img src="https://raw.githubusercontent.com/wiki/plusk01/teensyimu/assets/teensycoords.png" width="50%"/>

## Preliminaries

### Accelerometer

Recall from hw2-lab, the measurement model of the accelerometer is

$$
\mathbf{y}_\mathbf{a} = \mathbf{f} + \mathbf{b}_\mathbf{a} + \boldsymbol{\eta},
$$

where $\mathbf{f}=R^\text{b}_\text{w}(\mathbf{a} - \mathbf{g})$ is the specific force expressed in the body frame, $\mathbf{a}$ is the linear acceleration experienced by the accelerometer and $\mathbf{g}$ denotes the gravity vector, both expressed in the world coordinate frame. The $3\times 3$ matrix $R^\text{b}_\text{w}\in\text{SO}(3)$ encodes the *attitude*, or orientation, of the accelerometer w.r.t the world frame. Noise in accelerometer readings is captured with the zero-mean Gaussian process $\boldsymbol{\eta}(t)$ which has $\mathbb{E}[\boldsymbol{\eta}(t)\boldsymbol{\eta}(t+\tau)^\top]=\Sigma_\mathbf{a}\delta(\tau)$ and $\mathbf{b}_\mathbf{a}$, which is a slowly time-varying bias. Depending on the duration of the experiment, this bias can either be treated as a constant or as a random walk.

#### Accelerometers as Inclination Sensors

An accelerometer can be used to measure the static tilt angle (about x and y). Here, *static* emphasizes that the tilt angle is only valid when the accelerometer is not accelerating, i.e., the force of gravity is the only external force acting on the sensor. Using trigonometry, the tilt angles about x and y can be recovered given the accelerometer measurement. For example, see [here](https://www.digikey.com/en/articles/using-an-accelerometer-for-inclination-sensing) for figures, equations, and discussions.

### Gyroscopes

The angular rate $\boldsymbol{\omega}=\begin{bmatrix}p & q & r\end{bmatrix}^\top$ of the body of the teensyimu with respect to the inertial frame, expressed in the body frame, is measured by the rate gyro as

$$
\mathbf{y}_\boldsymbol{\omega} = \boldsymbol{\omega} + \mathbf{b}_\boldsymbol{\omega} + \boldsymbol{\nu},
$$

where $\boldsymbol{\nu}\sim\mathcal{N}(0,\Sigma_\boldsymbol{\omega})$ is the noise introduced in the measurement process and $\mathbf{b}_\boldsymbol{\omega}$ is a constant or slowly time-varying bias (i.e., $\dot{\mathbf{b}}_\boldsymbol{\omega}\approx0$).

Gyros typically have a sample rate of 500 Hz, 1 kHz, or 8 kHz.
They are great at capturing high-frequency dynamics and are incredibly useful, especially over short time windows.
However, due to the low-frequency bias $\mathbf{b}_\boldsymbol{\omega}$, simply integrating gyros over long timescales is unwise (e.g., more than tens of seconds).

### Attitude Kinematics

To develop a state dynamics equation, we need to understand how the angular rates and the tilt angles are related. The attitude of a rigid body evolves according to

$$
\dot{R} = R\omega_\times,
$$

where $R\in\text{SO}(3)$ is a $3\times 3$ matrix and $\omega_\times$ denotes a [skew-symmetric matrix](https://en.wikipedia.org/wiki/Skew-symmetric_matrix#Cross_product). This is the preferred form of integrating angular velocities and levarages the geometry of $\text{SO}(3)$, but adds complications for our simple tilt estimator. Instead, we will use [Euler angles](http://www.chrobotics.com/library/understanding-euler-angles). In this case, we can use the interaction matrix $\Gamma$ to write

$$
\frac{d}{dt}
\begin{pmatrix}
\phi \\ \theta \\ \psi
\end{pmatrix}
=
\begin{pmatrix}
1 & \sin\phi\tan\theta & \cos\phi\tan\theta \\
0 & \cos\phi & -\sin\phi \\
0 & \sin\phi\sec\theta & \cos\phi\sec\theta
\end{pmatrix}
\begin{pmatrix}
p \\ q \\ r
\end{pmatrix}
=
\Gamma \boldsymbol{\omega}.
$$

We can further simplify these kinematics by assuming small angles about level. Focusing only on roll $\phi$ and pitch $\theta$, we have

$$
\begin{align}
\dot{\phi} &\approx p \\
\dot{\theta} &\approx q
\end{align}
$$

We will use this state dynamics to design a linear Kalman filter for tilt estimation.

## Experimental Setup
Start with the cube sitting on a desk. Pick up the cube and move it in the air through any arbitrary trajectory for some time and then place it back on the desk on any one of it's six faces. The task is to do attitude estimation using a linear Kalman filter and predicting the final orientation of the cube (which face out of the six faces is up).

## Problem Statement

Design a Kalman filter that fuses gyro and accelerometer measurements and provides an estimate of the final orientation of the IMU cube. Using the approximate continuous-time attitude kinematics using Euler angles as discussed above, write the continuous-time linear system in state-space form as

$$
\begin{align}
\dot{\mathbf{x}} &= A\mathbf{x} + B\mathbf{u} + \mathbf{v} \\
\mathbf{y} &= C\mathbf{x} + \mathbf{w}
\end{align}
$$

with $\mathbb{E}[v(t)]=\mathbb{E}[w(t)]=0$, $\mathbb{E}[v(t)v(t+\tau)] = Q_c\delta(\tau)$ and $\mathbb{E}[w(t)w(t+\tau)] = R_c\delta(\tau)$. Note that we have not supplied any statistics for $\mathbf{v}$ or $\mathbf{w}$. In this case, we will use $Q$ and $R$ in the discrete Kalman filter simply as tuning knobs.

1. Discretize the continuous-time system.

2. Generate plots of the attitude evolution through time (Euler angles represented in the state) using only gyro integration (i.e., only the *propagation* step of the KF). Comment on the quality of this method for attitude estimation.

3. Generate plots of the attitude evolution (only roll and pitch angles for the accelerometer) using only the accelerometer. Comment on the quality of using only accelerometer measurements for attitude estimation.

4. Implement a Kalman filter (EKF) for attitude estimation, and plot the evolution of the estimate of the attitude (roll, pitch and yaw angles) as the measurements are processed by the Kalman filter. Comment if the filter is accurately able to predict which face of the cube is up at the end of the trajectory.

5. Compare the diagonal elements of the estimation error covariances and comment on the relative accuracy of the roll, pitch and yaw estimates.

## Advice
Start with a relatively simple trajectory to ensure that the attitude estimation code is working. Keep the trajectories short in duration since the drift due to gyroscope bias starts becoming significant. In particular, the predicted yaw angle deteriorates in quality for longer trajectories since gyroscope is the only source of information for yaw. In real world, periodic resets of the attitude (yaw) are carried out using other sensing modalities.

### Setup Code

**Note**: *The following code imports required packages and creates an IMU object which makes simplifies accessing data for real-time processing. There is no need to edit anything in this cell.*

In [ ]:
%gui qt
import time, sys

import numpy as np
np.set_printoptions(linewidth=130)

%matplotlib notebook
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 6]

import ipywidgets as widgets
from IPython.display import display

from jupyterplot import ProgressPlot # see https://github.com/lvwerra/jupyterplot

import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui

import esp32imu

class IMU:
    def __init__(self, Fs=100, port='/dev/ttyUSB0', baud=2000000):
        self.Fs = Fs # sample rate
        self.port = port
        self.baud = baud
        
        self.latestimu = None
        
        self._connect()
    
    def __del__(self):
        self._disconnect()
    
    def _connect(self):
        """
        Connect to the IMU via the esp32imu driver.
        
        A callback is registered that fires each time an IMU sample is received.
        """       
        self.driver = esp32imu.SerialDriver(self.port, self.baud) # start the serial driver to get IMU
        time.sleep(0.1) # requisite 100ms wait period for everything to get setup
        # Request a specific IMU sample rate.
        # the max sample rates of accel and gyro are 4500 Hz and 9000 Hz, respectively.
        # However, the sample rate requested is for the entire device. Thus, if a sample
        # rate of 9000 Hz is requested, every received data packet will have a new gyro
        # sample but repeated accelerometer samples
        self.driver.sendRate(self.Fs) # Hz
        time.sleep(0.1) # requisite 100ms wait period for everything to get setup

        # everytime an IMU msg is received, call the imu_cb function with the data
        self.driver.registerCallbackIMU(self._callback)
        
    def _disconnect(self):
        # make sure to clean up to avoid threading / deadlock errors
        self.driver.unregisterCallbacks()
    
    def _callback(self, msg):
        """
        IMU Callback

        Every new IMU sample received causes this function to be called.

        Parameters
        ----------
        msg : ti.SerialIMUMsg
            has fields: ['t_us', 'accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
        """
        # unpack data
        self.latestimu = {
            't': msg.t_us*1e-6,
            'acc': np.array([msg.accel_x, msg.accel_y, msg.accel_z]),
            'gyr': np.array([msg.gyro_x, msg.gyro_y, msg.gyro_z])
        }
        
    def reset(self):
        """
        Resets the IMU connection, which resets time
        """
        self._disconnect()
        self._connect()
        
    def get_time(self):
        """
        Get latest timestamp from IMU
        
        Returns
        -------
        t : float
            time (in seconds) of latest IMU measurement
        """
        return self.latestimu['t'] if 't' in self.latestimu else None
        
    def get_acc(self):
        """
        Get latest acceleration measurement from IMU
        
        Returns
        -------
        acc : (3,) np.array
            x, y, z linear acceleration
        """
        return self.latestimu['acc'] if 'acc' in self.latestimu else None
    
    def get_gyr(self):
        """
        Get latest gyro measurement from IMU
        
        Returns
        -------
        gyr : (3,) np.array
            x, y, z angular velocity
        """
        return self.latestimu['gyr'] if 'gyr' in self.latestimu else None


In [ ]:
def get_acc_angles(a):
    """
    Compute tilt angle (roll, pitch) from accelerometer data
    
    Parameters
    ----------
    a : (3,) np.array
        measurement vector from accelerometer
    
    Return
    ------
    y : (2,) np.array
        [ϕ, θ] in radians
    """
    
    ϕ = np.arctan2(a[1], np.sqrt(a[0]**2 + a[2]**2))
    θ = np.arctan2(-a[0], np.sqrt(a[1]**2 + a[2]**2))
    
    return np.array([ϕ, θ])

## Tilt Estimator - Your Code Here

**Note**: *If you use the following `while True` loop for real-time processing, you can end the loop by interrupting the kernel with the square 'stop' button in the toolbar.*

**Note**: *Due to memory and multiple `IMU` instances, measurements may become unreliable after many re-runs of the following cell. To prevent these oddities, regularly choose `Kernel > Restart & Run All` from the file menu.*